In [1]:
from credit_risk_analysis import features

2025-06-26 23:49:22.180 | INFO     | credit_risk_analysis.config:<module>:11 - PROJ_ROOT path is: C:\Users\andre\iCloudDrive\Documents_Backup\Digital_Bullet_Journal\Education\Data Science\Anyone AI\5. Final_Project\credit_risk_analysis


In [2]:
features.main()

Iniciando procesamiento de características...
✅ Archivos procesados y guardados correctamente.
Procesamiento completado.


In [2]:
features.main(stat_model_option='SelectKBest_f_classif', K=15)

2025-06-20 19:03:17.304 | INFO     | credit_risk_analysis.features:main:92 - Loading training and validation data...
2025-06-20 19:03:17.630 | SUCCESS  | credit_risk_analysis.features:main:101 - Loading training and validation data completed.
2025-06-20 19:03:18.419 | INFO     | credit_risk_analysis.features:main:108 - Setting up feature selection method...
2025-06-20 19:03:18.421 | SUCCESS  | credit_risk_analysis.features:main:120 - Feature selection method set to: SelectKBest(k=15)
2025-06-20 19:03:18.421 | INFO     | credit_risk_analysis.features:main:121 - Starting feature selection...
Selected Features: ['num__PAYMENT_DAY' 'num__MARITAL_STATUS' 'num__RESIDENCE_TYPE'
 'num__MONTHS_IN_RESIDENCE' 'num__FLAG_MASTERCARD'
 'num__QUANT_BANKING_ACCOUNTS' 'num__QUANT_SPECIAL_BANKING_ACCOUNTS'
 'num__QUANT_CARS' 'num__OCCUPATION_TYPE' 'num__MATE_PROFESSION_CODE'
 'num__EDUCATION_LEVEL.1' 'num__PRODUCT' 'num__AGE'
 'num__RESIDENCIAL_ZIP_3' 'num__PROFESSIONAL_ZIP_3']
2025-06-20 19:03:18.525 |

c:\Users\andre\iCloudDrive\Documents_Backup\Digital_Bullet_Journal\Education\Data Science\Anyone AI\5. Final_Project\final_project_anyoneai_venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [ 1  5 26 27 28 29] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\andre\iCloudDrive\Documents_Backup\Digital_Bullet_Journal\Education\Data Science\Anyone AI\5. Final_Project\final_project_anyoneai_venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


2025-06-20 19:03:19.278 | SUCCESS  | credit_risk_analysis.features:main:131 - df train/val selected features saved to: C:\Users\andre\iCloudDrive\Documents_Backup\Digital_Bullet_Journal\Education\Data Science\Anyone AI\5. Final_Project\credit_risk_analysis\data\processed\X_train|val_selected_features.csv


In [5]:
import pandas as pd

In [9]:
pd.read_csv('data/processed/X_train_selected_features.csv').head()

,num__PAYMENT_DAY,num__MARITAL_STATUS,num__RESIDENCE_TYPE,num__MONTHS_IN_RESIDENCE,num__FLAG_MASTERCARD,num__QUANT_BANKING_ACCOUNTS,num__QUANT_SPECIAL_BANKING_ACCOUNTS,num__QUANT_CARS,num__OCCUPATION_TYPE,num__MATE_PROFESSION_CODE,num__EDUCATION_LEVEL.1,num__PRODUCT,num__AGE,num__RESIDENCIAL_ZIP_3,num__PROFESSIONAL_ZIP_3
0,10.0,4.0,1.0,0.000000,0.0,1.0,1.0,1.0,0.0,0.000000,0.000000,2.0,55.0,171.0,171.0
1,5.0,1.0,1.0,5.000000,0.0,0.0,0.0,0.0,2.0,3.777494,0.294873,1.0,23.0,473.0,473.0
2,10.0,1.0,1.0,10.000000,0.0,0.0,0.0,0.0,2.0,3.777494,0.294873,1.0,27.0,726.0,726.0
3,10.0,2.0,2.0,5.000000,0.0,1.0,1.0,1.0,2.0,0.000000,0.000000,1.0,57.0,590.0,590.0
4,10.0,1.0,1.0,9.678929,1.0,0.0,0.0,0.0,1.0,3.777494,0.294873,1.0,50.0,235.0,235.0


In [14]:
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, ExtraTreesClassifier
#from sklearn.linear_model import LogisticRegression
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
#from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, f_classif
from sklearn.preprocessing import MinMaxScaler

In [5]:
import pandas as pd

In [7]:
X_train = pd.read_csv('data/interim/X_train.csv')
y_train = pd.read_csv('data/interim/y_train.csv')

In [ ]:
# Identify categorical and numerical columns
categorical_columns = X_train.drop(columns='CLERK_TYPE').select_dtypes(include=['object', 'category']).columns
numerical_columns = X_train.drop(columns='ID_CLIENT').select_dtypes(include=['int64', 'float64']).columns

# Define preprocessing for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values with mean for numerical features
            ('scaler', MinMaxScaler())  # Scale numerical features
        ]), numerical_columns),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with mode for categorical features
            #('encoder', OneHotEncoder(handle_unknown='ignore'))  # Encode categorical features
        ]), categorical_columns)
    ]
)

# Define the base estimator for RFE
base_estimator = ExtraTreesClassifier(random_state=42)

# Define the number of top features to select
k = 15

# Create a pipeline for preprocessing, feature selection, and model training
pipeline = Pipeline([
    ('preprocessing', preprocessor),  # Preprocess the data
    #('feature_selection', SelectKBest(score_func=mutual_info_classif, k=k)),  # Select top k features
    ('feature_selection', VarianceThreshold(threshold=0.01)),  # Remove low variance features
    ('classifier', ExtraTreesClassifier(random_state=42, class_weight='balanced_subsample'))  # Extra Trees Classifier
])

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train.values.ravel())

# Evaluate the pipeline on the validation set
y_pred = pipeline.predict(X_val)
print(classification_report(y_val, y_pred))

# Get the selected feature names
#selected_features = pipeline.named_steps['feature_selection'].get_support(indices=True)
#preprocessed_feature_names = pipeline.named_steps['preprocessing'].get_feature_names_out()
#selected_feature_names = preprocessed_feature_names[selected_features]
#print("Selected Features:", selected_feature_names)

ValueError: could not convert string to float: 'Web'

In [6]:
selected_features = pipeline.named_steps['feature_selection_variance'].get_support(indices=True)
preprocessed_feature_names = pipeline.named_steps['preprocessing'].get_feature_names_out()
selected_feature_names = preprocessed_feature_names[selected_features]
print("Selected Features:", selected_feature_names)

Selected Features: ['num__PAYMENT_DAY' 'num__MARITAL_STATUS' 'num__QUANT_DEPENDANTS'
 'num__NACIONALITY' 'num__RESIDENCE_TYPE' 'num__MONTHS_IN_RESIDENCE'
 'num__FLAG_EMAIL' 'num__PERSONAL_MONTHLY_INCOME' 'num__OTHER_INCOMES'
 'num__FLAG_VISA' 'num__FLAG_MASTERCARD' 'num__QUANT_BANKING_ACCOUNTS'
 'num__QUANT_SPECIAL_BANKING_ACCOUNTS' 'num__PERSONAL_ASSETS_VALUE'
 'num__QUANT_CARS' 'num__MONTHS_IN_THE_JOB' 'num__PROFESSION_CODE'
 'num__OCCUPATION_TYPE' 'num__MATE_PROFESSION_CODE'
 'num__EDUCATION_LEVEL.1' 'num__PRODUCT' 'num__AGE'
 'num__RESIDENCIAL_ZIP_3' 'num__PROFESSIONAL_ZIP_3'
 'cat__APPLICATION_SUBMISSION_TYPE_0'
 'cat__APPLICATION_SUBMISSION_TYPE_Carga'
 'cat__APPLICATION_SUBMISSION_TYPE_Web' 'cat__SEX_F' 'cat__SEX_M'
 'cat__STATE_OF_BIRTH_ ' 'cat__STATE_OF_BIRTH_AC' 'cat__STATE_OF_BIRTH_AL'
 'cat__STATE_OF_BIRTH_BA' 'cat__STATE_OF_BIRTH_CE'
 'cat__STATE_OF_BIRTH_ES' 'cat__STATE_OF_BIRTH_GO'
 'cat__STATE_OF_BIRTH_MA' 'cat__STATE_OF_BIRTH_MG'
 'cat__STATE_OF_BIRTH_MT' 'cat__STATE_O

In [7]:
len(selected_feature_names)

133

In [7]:
X_train.RESIDENCIAL_PHONE_AREA_CODE.value_counts()

RESIDENCIAL_PHONE_AREA_CODE
       5230
5      2914
97     1870
107    1864
54     1478
       ... 
106       1
64        1
60        1
8         1
59        1
Name: count, Length: 94, dtype: int64

In [12]:
X_train.describe().T

,count,mean,std,min,25%,50%,75%,max
ID_CLIENT,16384.0,24958.226562,14441.337506,5.0,12342.0,25016.5,37505.25,49989.0
PAYMENT_DAY,16384.0,12.887085,6.629545,1.0,10.0,10.0,15.00,25.0
QUANT_ADDITIONAL_CARDS,16384.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0
POSTAL_ADDRESS_TYPE,16384.0,1.006042,0.077501,1.0,1.0,1.0,1.00,2.0
MARITAL_STATUS,16384.0,2.161255,1.334400,0.0,1.0,2.0,2.00,7.0
QUANT_DEPENDANTS,16384.0,0.652954,1.233144,0.0,0.0,0.0,1.00,53.0
EDUCATION_LEVEL,16384.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0
NACIONALITY,16384.0,0.963196,0.198389,0.0,1.0,1.0,1.00,2.0
RESIDENCE_TYPE,15921.0,1.246404,0.861457,0.0,1.0,1.0,1.00,5.0
MONTHS_IN_RESIDENCE,15120.0,9.728042,10.720125,0.0,1.0,6.0,15.00,228.0
